该模块提供了类似于Perl中的正则表达式匹配操作。

模式和要搜索的字符串都可以是 Unicode 字符串（str）和 8 位字符串（byte）。然而，Unicode字符串和8位字符串不能混合使用：也就是说，你不能用字节模式匹配Unicode字符串，反之亦然；同样，当要求进行替换时，替换字符串必须与模式和搜索字符串的类型相同。

正则表达式使用反斜杠字符 ('\') 来表示特殊形式，或者允许使用特殊字符而不调用其特殊含义。这与 Python 在字符串字面意义上对同一字符的使用发生了冲突；例如，为了匹配一个字面意义上的反斜杠，我们可能要把 '\\\\' 写成模式字符串，因为正则表达式必须是 \\，而每个反斜杠必须在 Python 正则字符串字面意义内表示为 \\。另外，请注意，Python 在字符串字面意义上使用反斜杠时，任何无效的转义序列现在都会产生一个 DeprecationWarning，在将来，这将成为一个 SyntaxError。即使它是一个正则表达式的有效转义序列，这种行为也会发生。

解决办法是对正则表达式模式使用 Python 的原始字符串符号；在以 'r' 为前缀的字符串字面中，反斜线不会以任何特殊方式处理。所以 r"\n" 是一个包含 '\' 和 'n' 的两个字符的字符串，而 "\n" 是一个包含换行的一个字符的字符串。通常模式会在 Python 代码中使用这种原始的字符串符号来表达。

需要注意的是，大多数正则表达式操作都可以作为模块级的函数和方法在已编译的正则表达式上使用。这些函数是捷径，不需要你先编译一个 regex 对象，但缺少一些微调参数。

另请参见第三方regex模块，它的API与标准库re模块兼容，但提供额外的功能和更彻底的Unicode支持。



# 正则表达式语法
一个正则表达式（或 RE）指定了一组与之匹配的字符串；本模块中的函数让你检查一个特定的字符串是否与给定的正则表达式匹配（或者一个给定的正则表达式是否与一个特定的字符串匹配，这归根结底是同一件事）。

正则表达式可以串联起来形成新的正则表达式；如果A和B都是正则表达式，那么AB也是一个正则表达式。一般来说，如果一个字符串p匹配A，另一个字符串q匹配B，那么字符串pq将匹配AB。这一点是成立的，除非A或B包含低优先级的操作；A和B之间有边界条件；或者有编号的组引用。因此，复杂的表达式可以很容易地从较简单的原始表达式中构造出来，比如这里描述的表达式。关于正则表达式的理论和实现的细节，请参考Friedl的书[Frie09]，或者几乎所有关于编译器构造的教科书。

下面是对正则表达式格式的简要解释。如果想了解更多信息和更温和的介绍，请参考正则表达式 HOWTO。

正则表达式可以包含特殊和普通字符。大多数普通字符，像'A'、'a'或'0'，是最简单的正则表达式；它们只是简单地匹配自己。你可以将普通字符连接起来，所以last匹配字符串'last'。(在本节的其余部分，我们将用这种特殊的方式来写RE，通常不加引号，要匹配的字符串用 "单引号 "表示）。

有些字符，如'|'或'('，是特殊的。特殊字符要么代表普通字符的类别，要么影响它们周围的正则表达式的解释方式。

重复限定词（*、+、？、{m、n}等）不能直接嵌套。这可以避免与非贪婪的修饰词后缀 ? 以及其他实现中的其他修饰词产生歧义。为了将第二个重复应用于一个内部重复，可以使用括号。例如，表达式(?:a{6})*匹配六个'a'字符的任何倍数。

## 特殊字符是:

- .
(Dot.)在默认模式下，它匹配除换行以外的任何字符。如果指定了DOTALL标志，这将匹配包括换行在内的任何字符。



In [1]:
import re
m = re.search('...', 'abcdef')
m.group(0)

'abc'

- ^
(Cert.) 匹配字符串的开头，在MULTILINE模式下，也匹配每个换行之后的字符。

In [8]:
import re
m = re.search('^ab', 'abcdef')
m.group(0)

'ab'

- $
匹配字符串的末尾或字符串末尾的换行前，在MULTILINE模式下也匹配换行前。 foo同时匹配'foo'和'foobar'，而正则表达式foo$只匹配'foo'。更有趣的是，在'foo1\nfoo2\n'中搜索foo.$，正常情况下匹配'foo2'，但在MULTILINE模式下匹配'foo1'；在'foo\n'中搜索单个$会发现两个（空）匹配：一个在换行前，另一个在字符串的末尾。



In [12]:
import re
m = re.search('f$', 'abcdef')
m.group(0)

'f'

## *
*
导致产生的RE与前面的RE的0个或更多的重复相匹配，重复的次数越多越好。 ab*将匹配'a'、'ab'或'a'后面有任何数量的'b'。


In [13]:
import re
m = re.search('a*', 'aabcdef')
m.group(0)

'aa'

## +
导致产生的RE与前面的RE的1个或多个重复相匹配。 ab+将匹配'a'后面的任何非零数量的'b'；它不会只匹配'a'。


In [16]:
import re
m = re.search('ab+', 'aabbcdef')
m.group(0)

'abb'


## ?
导致产生的RE与前面的RE的0或1次重复相匹配。 ab? 将匹配'a'或'ab'。



In [21]:
import re
m = re.search('(a?)', 'aabbcdef')
m.group(0)
m.groups()

'a'

In [ ]:
- *?, +?, ??
'*'、'+'和'?'限定词都是贪婪的；它们尽可能多地匹配文本。有时这种行为并不可取；如果 RE <.*> 与 '<a> b <c>' 匹配，它将匹配整个字符串，而不仅仅是 '<a>' 。在限定词后面添加 ? 使它以非贪婪或最小的方式进行匹配；尽可能少的字符将被匹配。使用 RE <.*?> 将只匹配 '<a>'。

- {m}
指明应准确匹配前一个RE的m个副本；更少的匹配导致整个RE不被匹配。例如，a{6}将正好匹配六个'a'字符，而不是五个。

- {m,n}
导致产生的RE匹配前一个RE的m到n个重复，试图尽可能多地匹配重复的内容。例如，a{3,5}将匹配3到5个'a'字符。省略m是指下限为0，省略n是指无限的上限。例如，a{4,}b将匹配 "aaaab "或1000个 "a "字符后跟一个 "b"，但不匹配 "aaab"。逗号不能省略，否则修饰语会与之前描述的形式相混淆。

- {m,n}?
导致产生的RE与前面的RE的m到n次重复进行匹配，试图尽可能少地匹配重复。这是前一个限定词的非贪婪版本。例如，对于6个字符的字符串'aaaaa'，a{3,5}将匹配5个'a'字符，而a{3,5}? 将只匹配3个字符。

- \
可以转义特殊字符（允许你匹配'*'、'?'等字符），或者发出特殊序列的信号；特殊序列将在下面讨论。

如果你不是用一个原始的字符串来表达模式，请记住 Python 在字符串字面中也使用反斜杠作为转义序列；如果转义序列不被 Python 的分析器识别，那么反斜杠和随后的字符将包含在结果字符串中。然而，如果 Python 能识别结果序列，那么反斜线应该重复两次。这很复杂，也很难理解，所以强烈建议你除了最简单的表达式外，都使用原始字符串。

- []
用来表示一组字符。在一个集合中。

字符可以单独列出，例如，[amk]将匹配 "a"、"m "或 "k"。

字符的范围可以通过给出两个字符并以'-'分隔来表示，例如[a-z]将匹配任何小写ASCII字母，[0-5][0-9]将匹配所有从00到59的两位数，[0-9A-Fa-f]将匹配任何十六进制的数字。如果"-"被转义（例如：[a/z]），或者它被置于第一个或最后一个字符（例如：[-a]或[a-]），它将匹配字面的"-"。

特殊字符在集合内会失去其特殊意义。例如，[(+*)]将匹配任何字面字符'('、'+'、'*'、或')'。

诸如 \w 或 \S（定义见下文）这样的字符类别也可以在集合内接受，尽管它们匹配的字符取决于ASCII或LOCALE模式是否生效。

不在一个范围内的字符可以通过补充集来匹配。如果集合的第一个字符是'^'，所有不在集合内的字符都将被匹配。例如，[^5]将匹配除'5'以外的任何字符，而[^^]将匹配除'^'以外的任何字符。如果^不是集合中的第一个字符，则没有特殊含义。

要在一个集合内匹配一个字面的']'，在它前面加一个反斜杠，或者把它放在集合的开头。例如，[()[/]{}]和[]()[{}]都将匹配一个小括号。

未来可能会添加Unicode技术标准#18中对嵌套集和集合操作的支持。这将改变语法，所以为了促进这一变化，在不明确的情况下，将暂时发出 "未来警告"。这包括以文字'['开头或包含文字序列'--'、'&&'、'~~'和'||'的字符集。为了避免出现警告，可以用反斜杠转义。

在3.7版本中改变了：如果一个字符集包含的结构将在未来发生语义上的变化，就会发出FutureWarning。
    
- |
A|B，其中A和B可以是任意的RE，创建一个匹配A或B的正则表达式。任意数量的RE可以用'|'来分隔。这也可以在组内使用（见下文）。在扫描目标字符串时，由'|'分隔的RE从左到右被尝试。当一个模式完全匹配时，该分支被接受。这意味着一旦A匹配，B就不会被进一步测试，即使它能产生一个更长的整体匹配。换句话说，'|'运算符从不贪婪。要匹配一个字面意义上的'|'，请使用 \|，或者将其包含在一个字符类中，如 [|] 。

- (...)
匹配括号内的任何正则表达式，并表示一个组的开始和结束；一个组的内容可以在进行匹配后被检索到，并可以在字符串的后面用\number特殊序列进行匹配，如下所述。要匹配字面意义的'('或')'，请使用 \(或 \)，或将它们括在一个字符类中。[(], [)].

- (?...)
这是一个扩展符号（'('后面的'?'在其他方面没有意义）。'?'后面的第一个字符决定了该结构体的含义和进一步的语法是什么。扩展通常不会创建一个新的组；(?P<name>...)是这个规则的唯一例外。以下是目前支持的扩展。

- (?aiLmsux)
(一个或多个来自'a', 'i', 'L', 'm', 's', 'u', 'x'的字母。)该组匹配空字符串；这些字母设置了相应的标志：re.A（只匹配ASCII码），re.I（忽略大小写），re.L（依赖区域设置），re.M（多行），re.S（点匹配所有），re.U（Unicode匹配），和re.X（粗略），针对整个常规表达。(如果你希望将标志作为正则表达式的一部分，而不是向 re.compile() 函数传递一个标志参数，那么这很有用。标志应该在表达式字符串中首先使用。

- (?:...)
正则括号的非捕获版本。匹配括号内的任何正则表达式，但该组匹配的子串在执行匹配后不能被检索到，也不能在模式的后面被引用。

- (?aiLmsux-imsx:...)
(从'a', 'i', 'L', 'm', 's', 'u', 'x'这一组中选取零个或多个字母，后面可以选择'-'，然后是'i', 'm', 's', 'x'中的一个或多个字母。) 这些字母设置或移除相应的标志：re.A（仅ASCII码匹配），re.I（忽略大小写），re.L（与地区相关），re.M（多行），re.S（点匹配所有），re.U（Unicode匹配），以及re.X（粗略），针对表达式的部分。(这些标志在模块内容中描述）。

当作为内联标志使用时，字母'a'、'L'和'u'是相互排斥的，所以它们不能合并或跟在'-'后面。相反，当它们中的一个出现在一个内联组中时，它将覆盖包围组中的匹配模式。在Unicode模式中，(?a:...)切换到纯ASCII匹配，(?u:...)切换到Unicode匹配（默认）。在字节模式中，(?L:...)切换到根据区域划分的匹配，而(?a:...)切换到仅ASCII匹配（默认）。这种覆盖只对狭窄的内联组有效，而在组外则恢复原来的匹配模式。

3.6版中的新功能。

3.7版中的改变：字母'a'、'L'和'u'也可以在组中使用。

(?P<name>...)
类似于普通的小括号，但是由组匹配的子串可以通过符号化的组名名称访问。组名必须是有效的 Python 标识符，并且每个组名在正则表达式中只能定义一次。符号组也是一个有编号的组，就像该组没有被命名一样。

命名的组可以在三种情况下被引用。如果模式是(?P<quote>['"]).*?(?P=quote)（即匹配一个用单引号或双引号引用的字符串）。